In [1]:
import re
import pandas as pd
import numpy as np
import datetime

In [ ]:
d49 = today.strftime("SPUD%m%d.txt")

url49 = 'https://static.aer.ca/prd/data/wells/' + d49

In [2]:
def parser(file):

    lines = []

    with open(file, 'r') as file:
        for line in file:
            lines.append(line.rstrip('\n'))

    #Extract Date
    Date = (re.split('[\s]{4,}', lines[1].strip()))
    Date = Date[0]
    Date = Date.replace('Run Date: ', '')

    #Extract indices to cut list
    indices = [i for i, s in enumerate(lines) if '------' in s]
    indicest = [i for i, s in enumerate(lines) if 'TOTAL  -' in s]

    #Create list with only well licenses
    df1 = lines[(indices[1]+1):(indicest[0]-1)]

    dfsplit = lines[indices[1]]

    def find(s, ch):
        return [i for i, ltr in enumerate(s) if ltr == ch]

    cut = find(dfsplit,' ')

    df2 = []
    for line in df1:
        df2.append(line[0:cut[0]])
        df2.append(line[cut[0]:cut[1]])
        df2.append(line[cut[1]:cut[2]])
        df2.append(line[cut[2]:cut[3]])
        df2.append(line[cut[3]:cut[4]])
        df2.append(line[cut[4]:cut[5]])
        df2.append(line[cut[5]:cut[6]])
        df2.append(line[cut[6]:cut[7]])
        df2.append(line[cut[7]:cut[8]])
        df2.append(line[cut[8]:cut[9]])
        df2.append(line[cut[9]:cut[10]])
        df2.append(line[cut[10]:])

    #Create a list without all spaces
    df3 = []
    for line in df2:
        df3.extend(re.split('[\s]{5,}', line.strip()))

    df3 = [df3[x:x+12] for x in range(0, len(df3), 12)]

    #create a dataframe out of the list
    if df3 != []:
        data1 = pd.DataFrame(df3)
        data1.set_axis(['WELL ID', 'WELL NAME', 'LICENCE', 'CONTRACTOR BA ID', 'CONTRACTOR NAME', 'RIG NUMBER', 'ACTIVITY DATE', 'FIELD CENTRE', 'BA ID', 'LICENSEE', 'NEW PROJECTED TOTAL DEPTH', 'ACTIVITY TYPE'], axis = 1, inplace=True)
        data1['DATE'] = Date

        data1['ATS'] = data1['WELL ID'].str.replace(r'[^0-9]+', '')
        data1['ATS'] = data1['ATS'].str[-9:-1]
        data1['ATS'] = data1['ATS'].astype('int64')
        
        data1['DATE'] = data1['DATE'].astype('datetime64')
        data1['WEEK'] = data1['DATE'].dt.week
        data1['MONTH'] = data1['DATE'].dt.month
        data1['YEAR'] = data1['DATE'].dt.year

        convcomp = pd.read_csv('ConvComp.csv')
        convcomp['ConvDF'] = convcomp['ConvDF'].astype('int64')
        dfm = pd.merge(data1, convcomp, how='left', left_on='ATS', right_on='ConvDF')

    return dfm

In [3]:
import os

from pathlib import Path

directory_in_str = "ST49/"

directory = os.fsencode(directory_in_str) 

columns = ['WELL ID', 'WELL NAME', 'LICENCE', 'CONTRACTOR BA ID', 'CONTRACTOR NAME', 'RIG NUMBER', 'ACTIVITY DATE', 'FIELD CENTRE', 'BA ID', 'LICENSEE', 'NEW PROJECTED TOTAL DEPTH', 'ACTIVITY TYPE', 'DATE', 'ATS', 'ConvDF', 'Lat', 'Long', 'WEEK', 'MONTH']

df = pd.DataFrame(columns=columns)

ext = ['.txt','.TXT']

for file in os.listdir(directory): 
    filename = os.fsdecode(file) 
    if filename.endswith(tuple(ext)): 
        data = parser(os.path.join(directory_in_str, filename))
        df = df.append(data, ignore_index=True, sort=False)
        continue 
    else: 
        continue

In [4]:
df

,WELL ID,WELL NAME,LICENCE,CONTRACTOR BA ID,CONTRACTOR NAME,RIG NUMBER,ACTIVITY DATE,FIELD CENTRE,BA ID,LICENSEE,NEW PROJECTED TOTAL DEPTH,ACTIVITY TYPE,DATE,ATS,ConvDF,Lat,Long,WEEK,MONTH,YEAR
0,00/16-22-041-11W5/0,PEYTO CHAMBERS 16-22-41-11,0497802,A08C,Savanna Drilling Corp.,616,13 Mar 2020 08:45:00 PM,Red Deer,0YA2,Peyto Exploration & Development Corp.,,Drill To LD,2020-03-14,22041115,22041115,52.544925,-115.488797,11,3,2020.0
1,02/07-05-050-19W5/0,TOURMALINE HZ 102 ANSELL 7-5-50-19,0496917,A6PR,Nabors Drilling Canada Limited,C34,14 Mar 2020 01:00:00 AM,Drayton Valley,A573,Tourmaline Oil Corp.,,Resumption,2020-03-14,5050195,5050195,53.286973,-116.731900,11,3,2020.0
2,02/04-23-052-07W4/0,CREW HZ 102 SWIMMING 4-23-52-7,0497817,0Y52,Precision Drilling Corporation,299,14 Mar 2020 04:30:00 PM,Wainwright,A0C6,Crew Energy Inc.,,Drill To LD,2020-03-14,23052074,23052074,53.504319,-110.915807,11,3,2020.0
3,00/13-10-062-19W5/0,CHEVRON HZ FOXCK 13-10-62-19,0492892,0Z41,Ensign Drilling Inc.,144,14 Mar 2020 05:10:00 PM,St. Albert,0045,Chevron Canada Limited,,Drill To LD,2020-03-14,10062195,10062195,54.348600,-116.751569,11,3,2020.0
4,00/05-18-066-05W6/0,PARA HZ KARR 5-18-66-5,0489282,A5DR,Fox Drilling Inc.,3,06 Apr 2020 08:04:00 AM,Grande Prairie,0AW4,Paramount Resources Ltd.,,Drill To LD,2020-04-06,18066056,18066056,54.712582,-118.730402,15,4,2020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1358,AB/11-04-092-08W4/0,SUNCOR LEWIS 11-4-92-8,0497293,0Z41,Ensign Drilling Inc.,402,06 Feb 2020 11:30:00 PM,Bonnyville,0054,Suncor Energy Inc.,,Resumption,2020-02-07,4092084,4092084,56.951818,-111.213149,6,2,2020.0
1359,AC/14-12-092-08W4/0,SUNCOR LEWIS 14-12-92-8,0497298,0ZF6,McAllister Water Well Drilling Ltd.,1,06 Feb 2020 09:00:00 PM,Bonnyville,0054,Suncor Energy Inc.,,Set Surface,2020-02-07,12092084,12092084,56.966240,-111.132687,6,2,2020.0
1360,AA/04-17-092-08W4/0,SUNCOR LEWIS 4-17-92-8,0497302,0Z41,Ensign Drilling Inc.,406,07 Feb 2020 11:00:00 AM,Bonnyville,0054,Suncor Energy Inc.,,Resumption,2020-02-07,17092084,17092084,56.980894,-111.239956,6,2,2020.0
1361,AA/11-21-095-06W4/0,SUNCOR FIREBAG 11-21-95-6,0496572,A57H,Predator Drilling Inc.,3,07 Feb 2020 05:30:00 AM,Bonnyville,0054,Suncor Energy Inc.,,Resumption,2020-02-07,21095064,21095064,57.257203,-110.900608,6,2,2020.0


Export to CSV

In [5]:
export = df.to_csv (r'ST49.csv', index = False)